In [1]:

import os
import sys

# Pfad zum übergeordneten Verzeichnis ermitteln (einen Ordner zurück)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Das übergeordnete Verzeichnis zum Suchpfad hinzufügen
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

print(f"Das Verzeichnis '{parent_dir}' wurde zum sys.path hinzugefügt.")
#from pathlib import Path
from kg_manager_new import KGManager
# 1. Instanziieren
kg_manager = KGManager(ttl_path=r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test2_nachKGLoader.ttl", debug=True, case_sensitive=False)

# 2. Laden
kg_manager.load()

# 3. Analyse durchführen (Erkennt Calls & Assignments)
kg_manager.analyze_calls()

# 4. Speichern (überschreibt Standard, oder neuen Pfad angeben)
kg_manager.save(r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test2_filled.ttl")

Das Verzeichnis 'd:\MA_Python_Agent' wurde zum sys.path hinzugefügt.
Lade Graphen: D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test2_nachKGLoader.ttl
Graph geladen: 11525 Tripel.
Bereinige alte Analyse-Ergebnisse...
Bereinigung abgeschlossen. 0 Knoten entfernt.
Starte POU Call Analyse (Case Sensitive: False)...
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN] Port 'RESET1' an 'RS' nicht gefunden.
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN] Port 'RESET1' an 'RS' nicht gefunden.
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN] Port 'RESET1' an 'RS' nicht gefunden.
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN] Port 'RESET1' an 'RS' nicht gefunden.
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN] Port 'RESET1' an 'RS' nicht gefunden.
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN] Port 'RESET1' an 'RS' nicht gefunden.
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN] Port 'RESET1' an 'RS' nicht gefunden.
  [WARN] Port 'SET' an 'RS' nicht gefunden.
  [WARN]

WindowsPath('D:/MA_Python_Agent/MSRGuard_Anpassung/KGs/Test2_filled.ttl')

In [5]:
## Abfrage Konsistenz der Skills!! NICHT AUSFÜHREN
from rdflib import Graph

KG_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test2_filled.ttl"

g = Graph()
g.parse(KG_PATH, format="turtle")

Q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ap:  <http://www.semanticweb.org/AgentProgramParams/>
PREFIX dp:  <http://www.semanticweb.org/AgentProgramParams/dp_>
PREFIX op:  <http://www.semanticweb.org/AgentProgramParams/op_>

SELECT *
WHERE {
  ?var rdf:type ap:class_Variable ;
       dp:hasHardwareAddress ?addr ;
       op:isBoundToChannel ?ch .

  FILTER(
      ( REGEX(STR(?addr), "^(%?Q|Q )", "i") )
  )
}
ORDER BY ?var
"""

rows = []
for r in g.query(Q):
    rows.append({
        "var": str(r.var),
        "addr": str(r.addr),
        "ch": str(r.ch),
    })
    print(r.var, r.addr, r.ch)

#print("Mismatches:", len(rows))
#print(rows)



http://www.semanticweb.org/AgentProgramParams/GVL_HRL__dot__HRL_K1_1_Zaehler_Zuruecksetzen %Q* http://www.semanticweb.org/AgentProgramParams/IOChannel_TIID__dach__Device__leerz__1__leerz__(EtherCAT)__dach__Term__leerz__1__leerz__(EK1200)__dach__Term__leerz__2__leerz__(EL1512)__dach__CNT__leerz__Outputs__leerz__Channel__leerz__1__dach__Control__dach__Set__leerz__counter
http://www.semanticweb.org/AgentProgramParams/GVL_HRL__dot__HRL_K1_1_Zaehler_Zuruecksetzen Q 39.2 http://www.semanticweb.org/AgentProgramParams/IOChannel_TIID__dach__Device__leerz__1__leerz__(EtherCAT)__dach__Term__leerz__1__leerz__(EK1200)__dach__Term__leerz__2__leerz__(EL1512)__dach__CNT__leerz__Outputs__leerz__Channel__leerz__1__dach__Control__dach__Set__leerz__counter
http://www.semanticweb.org/AgentProgramParams/GVL_HRL__dot__HRL_K1_2_Zaehler_Zuruecksetzen %Q* http://www.semanticweb.org/AgentProgramParams/IOChannel_TIID__dach__Device__leerz__1__leerz__(EtherCAT)__dach__Term__leerz__1__leerz__(EK1200)__dach__Term__le